In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import csv
import re
import string
%matplotlib inline

import warnings

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


In [2]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))


In [5]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
! pip install wordcloud
from wordcloud import WordCloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
messages = pd.read_csv('/content/drive/MyDrive/AML/SMSSpamCollection',sep='\t',header=None)

In [9]:
messages

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
col_name = ["label", "message"]
messages.columns= col_name
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
messages.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [12]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.3 MB/s eta 0:00:00


In [13]:
import re
import contractions
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string 

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
def preprocessing(sentence):

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))

    punct = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
    tokenize = RegexpTokenizer(r'\w+')
  
    sentence = re.sub(punct, "", sentence)
    sentence = contractions.fix(sentence)
    sentence = re.sub(r'https?', "", sentence) 
    sentence = sentence.lower()
    sentence = tokenize.tokenize(sentence)

    sentence = [word for word in sentence if word not in stop_words]
    sentence = [str(lemmatizer.lemmatize(word)) for word in sentence]

    return(sentence)

In [15]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [16]:
labels = [0  if x== "ham" else 1 for x in messages.label]

In [17]:
X = [preprocessing(sentence) for sentence in messages.message]

In [18]:
data = pd.DataFrame({'message': X,'labels': labels})

In [35]:
def testTrainValSplit(file, seed):

  from sklearn.model_selection import train_test_split

  X_train, X_test, y_train, y_test = train_test_split(file.message, file.labels, test_size = 0.2, random_state = seed)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = seed)

  train=pd.DataFrame({'X_train': X_train,'y_train': y_train})
  val=pd.DataFrame({'X_val': X_val,'y_val': y_val})
  test=pd.DataFrame({'X_test': X_test,'y_test': y_test})

  train.to_csv('data/train.csv',index=False)
  val.to_csv('data/validation.csv',index=False)
  test.to_csv('data/test.csv',index=False)

In [36]:
data

,message,labels
0,"[go, jurong, point, crazy, available, bugis, n...",0
1,"[ok, lar, joking, wif, oni]",0
2,"[free, entry, 2, wkly, comp, win, fa, cup, fin...",1
3,"[dun, say, early, hor, c, already, say]",0
4,"[nah, think, go, usf, life, around, though]",0
...,...,...
5567,"[2nd, time, tried, 2, contact, 750, pound, pri...",1
5568,"[ü, b, going, esplanade, fr, home]",0
5569,"[pity, mood, soany, suggestion]",0
5570,"[guy, bitching, acted, like, id, interested, b...",0


In [37]:
testTrainValSplit(data, 11)

In [38]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

train

,X_train,y_train
0,"['hey', 'babe', 'doin', 'wot', '2', '2nite', '...",0
1,"['check', 'choose', 'babe', 'video', 'smsshsex...",1
2,"['bull', 'plan', 'go', 'floating', 'ikea', 'wi...",0
3,"['fine', 'miss', 'much']",0
4,"['honeymoon', 'outfit']",0
...,...,...
4006,"['got', 'take', '2', 'take', 'part', 'wrc', 'r...",1
4007,"['quite', 'know', 'still', 'cannot', 'get', 'h...",0
4008,"['come', 'people']",0
4009,"['sm', 'ac', 'blind', 'date', '4u', 'rodds1', ...",1


In [39]:
testTrainValSplit(data, 0)

In [40]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

train

,X_train,y_train
0,"['hi', 'darlin', 'finish', '3', '1', '2', 'pic...",0
1,"['freemsg', 'hi', 'baby', 'wow', 'got', 'new',...",1
2,"['talk', 'ever', 'ok', 'word']",0
3,"['take', 'care', 'financial', 'problemi', 'help']",0
4,"['might', 'ax', 'well']",0
...,...,...
4006,"['dear', 'uve', 'invited', 'xchat', 'final', '...",1
4007,"['oh', 'love', 'soooo', 'good', 'hear', 'omg',...",0
4008,"['get', 'gandhipuram', 'walk', 'cross', 'cut',...",0
4009,"['leaving', 'house']",0


## DVC 

In [41]:
!pip install dvc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
! pip install dvc_gdrive 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
!git init

Reinitialized existing Git repository in /content/.git/


In [44]:
!dvc init

ERROR: failed to initiate DVC - '.dvc' exists. Use `-f` to force.


In [45]:
!dvc add data

⠙ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
                                                                   
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
                                                                   
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
!
  0%|          |Transferring                          0/? [00:00<?,     ?file/s]
Transferring:   0% 0/3 [00:00<?, ?file/s{'info': ''}]                           
  0%|          |Transferring                    3/? [00:00<00:00, 1993.81file/s]

!

  0%|          |memory://.SnaCuS6ECWoMP9vsj6N2b4.tm0.00/? [00:00<?,        ?B/s]

memory://.SnaCuS6ECWoMP9vsj6N2b4.tmp:   0% 0.00/211 [00:00<?, ?B/s{'info': ''}] 

                                                                               
                                                                         

In [46]:
! ls data

test.csv  train.csv  validation.csv


In [47]:
! git add .gitignore
! git commit -m 'start tracking data using dvc'


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@32350c72ba9a.(none)')


In [48]:
! git add data.dvc

In [49]:
! git config --global user.email "bharathravilla349@gmail.com"

In [50]:
! git config --global user.name "Bharath349"

In [52]:
! git add .gitignore 
! git commit -m 'start tracking data using dvc'

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	drive/
	sample_data/
	test.csv
	train.csv
	validation.csv

nothing added to commit but untracked files present (use "git add" to track)


In [53]:
! dvc checkout

Checkout:   0% 0/3 [00:00<?, ?file/s{'info': ''}]  
!
Building data objects from data          |0.00 [00:00,      ?obj/s]


In [54]:
! git log

commit be1f8a8c914e8c56623ee116f146a2bb0fd1bbfa (HEAD -> master)
Author: Bharath349 <bharathravilla349@gmail.com>
Date:   Sat Apr 29 09:01:16 2023 +0000

    start tracking data


In [55]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

In [56]:
valid

,X_val,y_val
0,"['r', 'sure', 'understand', 'wine', 'good', 'i...",0
1,"['wot', 'student', 'discount', 'get', 'book']",0
2,"['boy', 'loved', 'gal', 'propsd', 'bt', 'mind'...",0
3,"['going', 'dinnermsg']",0
4,"['pas', 'dis', 'contact', 'n', 'see', 'wat', '...",0
...,...,...
441,"['anyways', 'go', 'gym', 'whatever', 'love', '...",0
442,"['hi', 'dear', 'saw', 'dear', 'happy', 'batter...",0
443,"['kyou', 'girl', 'waiting', 'reception', 'ah']",0
444,"['ever', 'one', 'foot']",0


In [57]:
## checking the data profile before update
print("Train set - number of 0s:", sum(train['y_train'] == 0), "number of 1s:", sum(train['y_train'] == 1))
print("Validation set - number of 0s:", sum(valid['y_val'] == 0), "number of 1s:", sum(valid['y_val'] == 1))
print("Test set - number of 0s:", sum(test['y_test'] == 0), "number of 1s:", sum(test['y_test'] == 1))

Train set - number of 0s: 3477 number of 1s: 534
Validation set - number of 0s: 393 number of 1s: 53
Test set - number of 0s: 955 number of 1s: 160


In [58]:
#with new seed 
testTrainValSplit(data, 444)

In [59]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

In [60]:
## checking the data profile before update
print("Train set - number of 0s:", sum(train['y_train'] == 0), "number of 1s:", sum(train['y_train'] == 1))
print("Validation set - number of 0s:", sum(valid['y_val'] == 0), "number of 1s:", sum(valid['y_val'] == 1))
print("Test set - number of 0s:", sum(test['y_test'] == 0), "number of 1s:", sum(test['y_test'] == 1))
     

Train set - number of 0s: 3459 number of 1s: 552
Validation set - number of 0s: 389 number of 1s: 57
Test set - number of 0s: 977 number of 1s: 138


In [61]:
! dvc add data
! git add data.dvc
! git commit -m 'changed random seed from 0 to 11'


⠙ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
                                                                   
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
                                                                   
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
!
  0%|          |Transferring                          0/? [00:00<?,     ?file/s]
Transferring:   0% 0/3 [00:00<?, ?file/s{'info': ''}]                           
  0%|          |Transferring                     3/? [00:00<00:00, 982.35file/s]

!

  0%|          |memory://.mT6KuUT3xiU6z4janvnEGi.tm0.00/? [00:00<?,        ?B/s]

memory://.mT6KuUT3xiU6z4janvnEGi.tmp:   0% 0.00/211 [00:00<?, ?B/s{'info': ''}] 

                                                                               
                                                                         

In [62]:
## getting previous version of data

! git checkout HEAD^ data.dvc

Updated 1 path from 4fa68da


In [63]:
! dvc pull data.dvc

Checkout:   0% 0/3 [00:00<?, ?file/s{'info': ''}]  
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
Checkout:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
  0%|          |/content/.dvc/cache/03/89f6e46df1060.00/? [00:00<?,        ?B/s]
  0% 0.00/36.6k [00:00<?, ?B/s{'info': ''}]                                     
Checkout: 100% 1/1 [00:00<00:00, 73.37file/s{'info': ''}]
!
  0%|          |/content/.dvc/cache/9f/3e5fe6e297b80.00/? [00:00<?,        ?B/s]
  0% 0.00/94.3k [00:00<?, ?B/s{'info': ''}]                                     
  0% Checkout|          |2/? [00:00<00:00, 126.25file/s] 
!
  0%|          |/content/.dvc/cache/53/96c4c18650da0.00/? [00:00<?,        ?B/s]
  0% 0.00/329k [00:00<?, ?B/s{'info': ''}]                                      
M       data/
1 file modified


In [64]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

## checking the data profile before update
print("Train set - number of 0s:", sum(train['y_train'] == 0), "number of 1s:", sum(train['y_train'] == 1))
print("Validation set - number of 0s:", sum(valid['y_val'] == 0), "number of 1s:", sum(valid['y_val'] == 1))
print("Test set - number of 0s:", sum(test['y_test'] == 0), "number of 1s:", sum(test['y_test'] == 1))
     

Train set - number of 0s: 3477 number of 1s: 534
Validation set - number of 0s: 393 number of 1s: 53
Test set - number of 0s: 955 number of 1s: 160


In [65]:

! git checkout HEAD data.dvc      
! dvc checkout  

Updated 1 path from baa38c2
Checkout:   0% 0/3 [00:00<?, ?file/s{'info': ''}]  
!
Building data objects from data          |0.00 [00:00,      ?obj/s]
Checkout:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
  0%|          |/content/.dvc/cache/11/704fde413f670.00/? [00:00<?,        ?B/s]
  0% 0.00/333k [00:00<?, ?B/s{'info': ''}]                                      
Checkout: 100% 1/1 [00:00<00:00, 67.37file/s{'info': ''}]
!
  0%|          |/content/.dvc/cache/62/421b3ed9462b0.00/? [00:00<?,        ?B/s]
  0% 0.00/36.0k [00:00<?, ?B/s{'info': ''}]                                     
  0% Checkout|          |2/? [00:00<00:00, 118.58file/s] 
!
  0%|          |/content/.dvc/cache/b3/4bcd44df736c0.00/? [00:00<?,        ?B/s]
  0% 0.00/90.4k [00:00<?, ?B/s{'info': ''}]                                     
M       data/


In [66]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
valid = pd.read_csv('data/validation.csv')

## checking the data profile before update
print("Train set - number of 0s:", sum(train['y_train'] == 0), "number of 1s:", sum(train['y_train'] == 1))
print("Validation set - number of 0s:", sum(valid['y_val'] == 0), "number of 1s:", sum(valid['y_val'] == 1))
print("Test set - number of 0s:", sum(test['y_test'] == 0), "number of 1s:", sum(test['y_test'] == 1))
     


Train set - number of 0s: 3459 number of 1s: 552
Validation set - number of 0s: 389 number of 1s: 57
Test set - number of 0s: 977 number of 1s: 138
